In [21]:
import re
from urllib.parse import urljoin
import os
#from logging import getLogger, config

import unicodedata
from multiprocessing.dummy import Pool
from time import sleep
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [22]:
import re
from urllib.parse import urljoin
import os
#from logging import getLogger, config

import unicodedata
from multiprocessing.dummy import Pool
from time import sleep
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup, SoupStrainer

def strip_url(text):
    url = re.compile(r'https?://(www\.)?', re.UNICODE)
    new_url =  url.sub('', text).strip().strip('/')
    return new_url.split('/')[0]

def normalize_site(website):
    new_list = website.split('/', 3)
    return '/'.join(map(str,new_list[:3]))

def dedupl(mylist):
    return list(dict.fromkeys(mylist))

def find_nested_url(a_tag, tag= 'a', attr= 'href'):
    urls = []
    for x in a_tag:
        if x.has_attr(attr):
            urls.append(x[attr])
    return urls
            
def clean_response(base_site, num_pages: int, to_mantain:list, longer_urls=True, tag='a', article_url_lenght:int=30):
    print('Starting collecting urls for website {}'.format(base_site))
    urls = []
    last_urls = []
    for page in range(0, num_pages+1):
        site = base_site.format(page)
        response = requests.get(site)
        soup = BeautifulSoup(response.text, "html.parser")
        website = normalize_site(site)
        a_tag = soup.find_all(tag)
        urls = dedupl(find_nested_url(a_tag))  
        final_urls = [urljoin(website, url) for url in urls if not url.startswith(website)]
            
        if longer_urls == True:
            splitted_list = [string.rsplit('/',1) for string in final_urls]
            new_urls = list(filter(lambda string: len(string[1]) > article_url_lenght, splitted_list))
            final_urls = ['/'.join(url) for url in new_urls]
                
        res = dedupl(final_urls)
        result = [i for i in res if re.search("|".join(to_mantain), i)]
        last_urls.append(result)
    return flatten(last_urls)
        
        #except Exception as e:
           # print('Cannot collect urls from website {w} due to error {e}'.format(w=site, e=e))  
        
def extract_all_links(site, tag ='a'):
    html = requests.get(site).text
    soup = BeautifulSoup(html, 'html.parser').find_all(tag)
    links = [link.get('href') for link in soup]
    return links

def flatten(l):
    return [item for sublist in l for item in sublist]

def drop_containing(sites_list, drop_urls):
    return [i for i in sites_list if not re.search("|".join(drop_urls), i)]

def scrape_wilson(base_site, num_pages=416):
    urls = []
    for page in range(0, num_pages+1):
        site = base_site.format(page)
        all_links = extract_all_links(site)
        links = [urljoin(site, i) for i in all_links]
        links = list(dict.fromkeys(links))
        urls.append(links)
        
    res = list(dict.fromkeys(flatten(urls)))
    to_mantain = ['/article/', '/publication/','/blog-post/', '/event/']
    result = [i for i in res if re.search("|".join(to_mantain), i)]
    return result

def scrape_general(base_site:str, num_pages: int, to_mantain:list ):
    urls = []
    for page in range(1, num_pages+1):
        site = base_site.format(page)
        all_links = extract_all_links(site)
        links = [urljoin(site, i) for i in all_links]
        links = list(dict.fromkeys(links))
        urls.append(links)
        
    res = list(dict.fromkeys(flatten(urls)))
    result = [i for i in res if re.search("|".join(to_mantain), i)]
    return result

In [23]:
clean_response('https://www.findaphd.com/phds/biological-sciences/?10M780&PG={}',1,['https://www.findaphd.com/'])

Starting collecting urls for website https://www.findaphd.com/phds/biological-sciences/?10M780&PG={}


IndexError: list index out of range

In [5]:
extract_all_links('https://www.findaphd.com/phds/biological-sciences/?10M780&PG=2')

['https://www.findaphd.com',
 '/phds/discipline/',
 '/phds/subject/',
 '/phds/country/',
 '/phds/phd-research-project/?02g0',
 '/phds/phd-research-programme/?02w0',
 '/phds/',
 '/phds/biological-sciences/?10M780',
 '/phds/business-and-management/?10M7a0',
 '/phds/chemistry/?10M7c0',
 '/phds/geography/?10M7y0',
 '/phds/engineering/?10M7o0',
 '/phds/discipline/',
 '/phds/united-kingdom/?g0w900',
 '/phds/australia/?g00w20',
 '/phds/germany/?g0Mw00',
 '/phds/china/?g0wE10',
 '/phds/ireland/?g00y00',
 '/phds/country/',
 '/guides/what-is-a-phd',
 '/guides/phd-thesis',
 '/guides/phd-interview-questions',
 '/guides/writing-phd-research-proposal',
 '/guides/contacting-phd-supervisors',
 '/blog/',
 '/authors/',
 '/guides/advice/',
 '/guides/phd-funding-guide',
 '/guides/uk-phd-loans-scheme',
 '/guides/research-council-studentships',
 '/guides/graduate-teaching-assistantships',
 '/guides/international-phd-funding-uk',
 '/guides/funding/',
 '/guides/phd-study-in-canada',
 '/guides/phd-study-in-aus

In [25]:
html = requests.get('https://scholarshipdb.net/scholarships/Program-PhD?page=2', verify=False)
html1 = html.text
soup = BeautifulSoup(html1, 'html.parser').find_all(tag ='a')
links = [link.get('href') for link in soup]
print(links)

C:\Users\Dessy\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scholarshipdb.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[]


In [19]:
print(html1)


<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="no-js ie ie6 ieFail"> <![endif]-->
<!--[if IE 7 ]> <html lang="en" class="no-js ie ie7 ieFail"> <![endif]-->
<!--[if IE 8 ]> <html lang="en" class="no-js ie ie8"> <![endif]-->
<!--[if IE 9 ]> <html lang="en" class="no-js ie ie9"> <![endif]-->
<!--[if !(IE)]><!-->
<html lang="en" class="no-ie no-js">
<!--<![endif]-->
<head><title>
	Biological Sciences PhD Projects, Programmes & Scholarships
</title><meta name="referrer" content="always" />
<link rel="canonical" href="https://www.findaphd.com/phds/biological-sciences/?10M780&amp;PG=3" />
<meta name="description" content="FindAPhD. Search Funded PhD Projects, Programmes &amp; Scholarships in Biological Sciences. Search for PhD funding, scholarships &amp; studentships in the UK, Europe and around the world." /><meta name="keywords" content="PhD, Biological, Sciences, PhD program, PhD programme, PhD funding, PhD scholarships, PhD studentships" />

<script>(function (w, d, s, l, i) {